In [1]:
%run ../../import_envs.py
%run ../apg_hyperparams.py

CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:0')

probtorch: 0.0+5a2c637 torch: 1.3.0
inference method:apg, apg sweeps:10, epochs:500, sample size:10, batch size:20, learning rate:0.0001


In [2]:
from affine_transformer import Affine_Transformer
AT = Affine_Transformer(frame_pixels=FRAME_PIXELS, digit_pixels=DIGIT_PIXELS, CUDA=CUDA, DEVICE=DEVICE)
mnist_mean = torch.from_numpy(np.load('../mnist_mean.npy')).float()
mnist_mean = mnist_mean.cuda().to(DEVICE)

In [3]:
from apg_modeling import init_model
models, optimizer = init_model(frame_pixels=FRAME_PIXELS,
                               digit_pixels=DIGIT_PIXELS, 
                               num_hidden_digit=NUM_HIDDEN_DIGIT, 
                               num_hidden_coor=NUM_HIDDEN_COOR, 
                               z_where_dim=Z_WHERE_DIM, 
                               z_what_dim=Z_WHAT_DIM, 
                               CUDA=CUDA, 
                               DEVICE=DEVICE, 
                               LOAD_VERSION=None, 
                               LR=LR)

In [ ]:
from apg import APG
apg = APG(models=models, 
          AT=AT, 
          K=K, 
          D=D, 
          T=T, 
          mnist_mean=mnist_mean, 
          training=True)


train(APG=apg, 
      optimizer=optimizer, 
      data_path=data_path, 
      Train_Params=Train_Params,
      APG_STEPS=APG_STEPS,
      DEVICE=DEVICE,
      PATH=PATH)

In [ ]:
Save_models(models, PATH=PATH)